<a href="https://colab.research.google.com/github/cff100/MecFLu2/blob/main/Ventos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cdsapi

In [ ]:
import cdsapi

c = cdsapi.Client(url='https://cds.climate.copernicus.eu/api/v2',
                  key='331408:d8c35ac5-fae8-4f2e-90fe-b448d76e3140')

In [ ]:
#Eólica Offshore


c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': '1200',
        'year': '2023',
         'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            -27, -45, -20, -35,  # Coordenadas da Bacia de Campos e um pouco mais pegando a costa brasileira
        ],
    },
    'Era5_2022_875HPA_CAMPOS.nc')
import xarray as xr

titulo = "Marcação das Plataformas na Bacia de Campos Desativadas"

titulo_html = """<h1 align="center" style="font-family: verdana; font-size:16px; font-weight:bold; text-transform: uppercase;">{}</h1>""".format(titulo)

2024-08-22 16:01:06,586 INFO Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
INFO:cdsapi:Welcome to the CDS.
 As per our announcements on the Forum, this instance of CDS will soon be decommissioned.
 Please update your cdsapi package to a version >=0.7.0, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
 The current legacy system will be kept for a while, but we will reduce resources gradually until full decommissioning in September 2024.
2024-08-22 16:01:06,592 WARNING MOVE TO

In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.2 MB/s eta 0:00:00


In [ ]:
!pip install netcdf4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

# Carrega o arquivo NetCDF com os dados
# Use the default engine, as netcdf4 will be automatically selected if installed
ds = xr.open_dataset('Era5_2022_875HPA_CAMPOS.nc')

# Calcula a velocidade do vento (wspd) a partir das componentes u e v
u = ds['u']
v = ds['v']
wspd = (u**2 + v**2)**0.5

# Calcula a média da velocidade do vento para cada ponto de grade ao longo dos horários e dias, agrupando por estação do ano
wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

# Adiciona marcadores para as localizações das plataformas de petróleo e gás
platforms = {
    'NAMORADO 2 (PNA-2)' :(-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)' :(-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)' :(-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)' :(-22.4868, -40.09779),
    'PETROBRAS IX' :(-22.57358, -40.82192),
    'PETROBRAS XIX' :(-22.3927, -40.05438),
    'PETROBRAS XXXIII' :(-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)' :(-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)' :(-22.17535, -40.29147),
}

# Cria um DataFrame para armazenar os resultados
result_df = pd.DataFrame(index=platforms.keys(), columns=['DJF', 'MAM', 'JJA', 'SON'])

# Loop sobre as estações do ano
for season in ['DJF', 'MAM', 'JJA', 'SON']:
    for platform, (lat, lon) in platforms.items():
        # Obtém o valor da média da velocidade do vento para a plataforma e estação do ano específicos
        value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
        result_df.at[platform, season] = value

# Exibe o DataFrame
print(result_df)

# Fecha o arquivo NetCDF
ds.close()

result_df

                           DJF       MAM       JJA        SON
NAMORADO 2 (PNA-2)    9.950451  6.886537   8.53317  11.133941
PETROBRAS 26 (P-26)   9.620162  6.811495  8.609517  10.933599
PETROBRAS 32 (P-32)  10.032628  6.831916  8.644569  11.175437
PETROBRAS 37 (P-37)   9.620162  6.811495  8.609517  10.933599
PETROBRAS IX          9.916655  6.894263  8.435221  11.005264
PETROBRAS XIX         9.620162  6.811495  8.609517  10.933599
PETROBRAS XXXIII       9.70816  6.779782  8.620629  10.951965
VERMELHO 1 (PVM-1)   10.032628  6.831916  8.644569  11.175437
VERMELHO 2 (PVM-2)   10.032628  6.831916  8.644569  11.175437


,DJF,MAM,JJA,SON
NAMORADO 2 (PNA-2),9.950451,6.886537,8.53317,11.133941
PETROBRAS 26 (P-26),9.620162,6.811495,8.609517,10.933599
PETROBRAS 32 (P-32),10.032628,6.831916,8.644569,11.175437
PETROBRAS 37 (P-37),9.620162,6.811495,8.609517,10.933599
PETROBRAS IX,9.916655,6.894263,8.435221,11.005264
PETROBRAS XIX,9.620162,6.811495,8.609517,10.933599
PETROBRAS XXXIII,9.70816,6.779782,8.620629,10.951965
VERMELHO 1 (PVM-1),10.032628,6.831916,8.644569,11.175437
VERMELHO 2 (PVM-2),10.032628,6.831916,8.644569,11.175437
